<a href="https://colab.research.google.com/github/HajarGhizlaneML/machine_learning_project/blob/master/formatted_data_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DL_Chatbot_Project**
Dans ce projet nous avons essayé de créer un chatbot qui peut mener une conversation avec un être-humain en appliquant quelques méthodes et modèles de Deep Learning.

**Importer les librairies nécessaires pour le projet**

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pandas as pd

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

Afin de créer un chatbot capable de mener des conversation à peu près comme un  être-humain nous avons utiliser:


1.   The Cornell Movie-Dialogs Corpus :  dataset contient une grande collection de métadonnées de conversations extarites de script des films. 
2.   Quelques fichiers contenant des conversations  sur d'autres sites comme Kaggle qui semble utile pour ce projet.



In [0]:
df = pd.read_csv("dataset_chat.csv") 
df = df.dropna()
df = df.reset_index(drop=True)
df

,id,Question,Response
0,0,there .,where ?
1,1,you have my word . as a gentleman,you re sweet .
2,2,hi .,looks like things worked out tonight huh ?
3,3,you know chastity ?,i believe we share an art instructor
4,4,have fun tonight ?,tons
...,...,...,...
64299,64374,you are immature,what can i say? i'm sure i've seen that myself.
64300,64375,you are a loser,i have failed at many things i've tried.
64301,64376,you psycho,you are crazy.
64302,64377,you look more like,so you like jocks?


**Création  un fichier formatted chatbot data**

> Nous allons créer un fichier de données (format txt) dans lequel chaque ligne contient une phrase de requête séparée par des tabulations et une paire de phrases de réponse.




In [0]:
pairs=[]
for i in range(len(df)):
  l=[]
  l.append(df['Question'][i])
  l.append(df['Response'][i])
  pairs.append(l)

In [0]:
# Define path to new file
corpus=""
datafile = os.path.join(corpus, "formatted_chatbot_data.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in pairs:
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
def printLines(file, n=64289):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
printLines(datafile)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
b'the mamma s there too .\tyes the mamma is is there .\n'
b'yes the mamma is is there .\tgood night enzo good night jacques .\n'
b'jacques ! johana !\tenzo !\n'
b'what are you doing here ?\tchecking out the competition .\n'
b'checking out the competition .\tenzo why do you guys do this ?\n'
b'enzo why do you guys do this ?\twhat else is there ?\n'
b'what else is there ?\tyou know what i m talking about .\n'
b'because you d laugh .\ti won t i swear .\n'
b'. . . not yet .\tdon t .\n'
b'don t .\twhy ?\n'
b'five minutes and fifty seconds .\tthat idiot !\n'
b'ten thousand .\tlira ?\n'
b'lira ?\tdollars .\n'
b'roberto ? how long ?\tsix seconds .\n'
b'have the car painted .\tsantini will do that for dollars !\n'
b'santini will do that for dollars !\tthen tell him to wax it too .\n'
b'yes ?\tfind the frenchman . find me mayol .\n'
b'enzo wake up !\teh ? si ! avanti !\n'
b'is he here ?\the s training .\n'
b'plenty of 